In [2]:
!pip install pymysql
!pip install json

#file name : dataFetchServices.py 
import pymysql 
import pandas as pd
from os import path
import json
from sklearn.model_selection import train_test_split
import os




def setup_connection():
    user = 'admin'
    DB_PASSWORD = 'HeyMultilex9087'
    DB_PORT = 3306
    passw = DB_PASSWORD
    host =  'multilex-db.csgyi8splofr.ap-south-1.rds.amazonaws.com'
    port = DB_PORT
    database = 'preipo'
    conn = pymysql.connect(host=host,port=port,user=user,passwd=passw,db=database)
    return conn

def fetch_data():
    conn = setup_connection()
    df = pd.read_sql('SELECT * FROM preipo.Multilex', con=conn)
    return df
def toJson(csvFilePath):
    # read csv file and create a dataframe DF 
    Df = pd.read_csv(csvFilePath, na_filter=False, dtype=str)
    # concatenate title and text of csv file  and out it in context in df
    Df["context"] = Df.title.str.cat(Df["text"], sep=" ")
    # drop text and title from DF
    Df.drop(["text", "title"], axis=1)
    # keep only context and companies in Df 
    Df = Df[["context",  "Companies"]]
    # make everything in context in lower case 
    Df["context"] = Df["context"].apply(lambda x:str(x).lower())
    # make everything in companies in lower case 

    Df["Companies"] = Df["Companies"].apply(lambda x:str(x).lower())
    # parent_dict = {"root":}
    data = []
    # quetions whose answer to be trained using multilex table 
    questions = ["which company is going ipo?", "which company is going to be listed?",
                 "which company is going public?"]
    for index, row in Df.iterrows():
         # create dictionary with key1- qas  as a iist and  context key as string 
        newDict = {"qas": [], "context": str(row["context"])}
        id = 1
        for question in questions:
             # for each question in questions create a tempDIct dctionary with
             # key1- id , key2-is_impossible - question is impossible to get from context
             # key3- quesiotn
            tempDict = {
                "id": str(id),
                "is_impossible": False,
                "question": question
            }
            # create a key4 -answers in tempDict 
            # answers is also a Dict with the below :
                #key1 - text- take company name from df object based on multiulex table 
                # key2- answer_start - find the position of the company string  in the  text of context column in df which is created out of title and text of multilex table 
            tempDict["answers"] = [
                {
                    "text": row["Companies"],
                    "answer_start": str(row["context"]).lower().find(str(row["Companies"]).lower())
                }
            ]
            # if we get a match inside the context for the company name 
              # append the temDict object to newDict object created in the beginning 
              #increment the id 
            if str(row["context"]).lower().find(str(row["Companies"])) != -1:
                newDict["qas"].append(tempDict)
                id += 1

        '''
        Example of data created using above logic 
        "qas": [
                        {
                            "answers": [
                                {
                                    "answer_start": 2574,
                                    "text": "donasi yang bisa"
                                }
                            ],
                            "id": "1",
                            "is_impossible": false,
                            "question": "which company is going ipo?"
                        },
                        {
                            "answers": [
                                {
                                    "answer_start": 2574,
                                    "text": "donasi yang bisa"
                                }
                            ],
                            "id": "2",
                            "is_impossible": false,
                            "question": "which company is going to be listed?"
                        },
                        {
                            "answers": [
                                {
                                    "answer_start": 2574,
                                    "text": "donasi yang bisa"
                                }
                            ],
                            "id": "3",
                            "is_impossible": false,
                            "question": "which company is going public?"
                        }
                    ]
                },
                {
                    "context": "want to ipo, estee gold feet euro market offer price rp 60 rp 70 per share reporter yuliana hema editor tendi mahadi kontan co id jakarta a prospective non cyclical issuer, pt estee gold feed tbk euro is ready to take the floor on the indonesia stock exchange through an initial public offering ipo the company, which is engaged in cosmetic aerosol filling services and household cleaning materials, offers a maximum of 500 million shares this value is equivalent to 20% of the company s issued and paid up capital after the ipo the shares are new shares issued from the portfolio with a nominal value of rp 5 per share of the total shares, estee gold feed offers an ipo price in the range of rp 60 to rp 70 per share thus, euro has the potential to reap fresh funds of up to rp 35 billion read also kusuma kemindo sentosa kkes targets ipo funds of up to idr 58 5 billion not only that, euro also issues a maximum of 500 series i warrants, equivalent to 25% of the total shares that have been assigned and fully paid these warrants will be given free of charge to shareholders whose names are recorded in the register of shareholders on the allotment date later, each holder of 1 one new share of estee gold feed will be entitled to 1 one series i warrant opex in its ipo celebration, estee gold feed appointed pt danatama makmur sekuritas to act as the underwriter next up facebook, whatsapp, instagram, netflix already registered pse, google, youtube haven t checked other news and articles on google news donate, get free vouchers your support will increase our enthusiasm to provide quality and useful articles as an expression of gratitude for your attention, there is a free voucher worth a donation that can be used for shopping at the kontan store support content news index tag ipo ipo plan ipo reporter yuliana hema editor tendi mahadi",
                      
                      # second set for the second company 
                              "qas": [
                        {
                            "answers": [
                                {
                                    "answer_start": 173,
                                    "text": "pt estee gold feed tbk"
                                }
                            ],
                            "id": "1",
                            "is_impossible": false,
                            "question": "which company is going ipo?"
                        },
                        {
                            "answers": [
                                {
                                    "answer_start": 173,
                                    "text": "pt estee gold feed tbk"
                                }
                            ],
                            "id": "2",
                            "is_impossible": false,
                            "question": "which company is going to be listed?"
                        },
                        {
                            "answers": [
                                {
                                    "answer_start": 173,
                                    "text": "pt estee gold feed tbk"
                                }
                            ],
                            "id": "3",
                            "is_impossible": false,
                            "question": "which company is going public?"
                        }
                    ]
                },
                {
                    "context": "daftar ipo emiten 2022 targetnya ada 55 perusahaan bisnis com jakarta bursa efek indonesia menargetkan bakal ada 55 intial public offering ipo atau penawaran saham perdana dari emiten baru tahun ini siapakah mereka semua sampai dengan 14 januari 2022 bei telah mengantongi 30 nama perusahaan yang bakal tercatat tahun ini direktur penilaian perusahaan bei i gede nyoman yetna mengatakan dari 30 perusahaan tersebut empat perusahaan merupakan perusahaan beraset kecil dengan nilai aset di bawah rp50 miliar lalu 14 perusahaan merupakan perusahaan aset skala menengah dengan nilai aset antara rp50 miliar hingga rp250 miliar kemudian 12 perusahaan merupakan perusahaan beraset besar dengan nilai aset di atas rp250 miliar hingga saat ini terdapat 30 perusahaan dalam daftar antrian pipeline pencatatan saham bei sebagai informasi berikut rincian sektor calon emiten yang akan ipo tahun ini 4 perusahaan dari sektor industrials 4 perusahaan dari sektor consumer non cyclicals 9 perusahaan dari sektor consumer cyclicals 4 perusahaan dari sektor technology 1 perusahaan dari sektor healthcare 2 perusahaan dari sektor energy 1 perusahaan dari sektor financials 3 perusahaan dari sektor properties real estate 2 perusahaan dari sektor infrastructures sementara itu nyoman menambahkan telah tercatat dua perusahaan yang mencatatkan saham di bei dengan total dana yang berhasil dihimpun rp723 miliar yaitu pt adaro minerals indonesia tbk admr dan pt semacom integrated tbk sema selain kedua perusahaan masih ada beberapa emiten yang kini dalam penawaran misalnya pt champ resto indonesia tbk enak pt net visi media tbk netv pt nusatama berkah tbk ntbk dan pt mitra angkasa sejahtera tbk baut di sisi lain otoritas jasa keuangan ojk menargetkan penghimpunan dana di pasar modal mencapai rp125 triliun rp175 triliun pada 2022 target tersebut lebih rendah dibandingkan realisasi penghimpunan dana di pasar modal mencapai rp363 3 triliun pada 2021 berdasarkan data ojk penghimpunan dana di pasar modal terdiri dari penawaran umum perdana saham initial public offering ipo senilai rp61 7 triliun yang terdiri dari 56 emiten baru penerbitan hak dengan memesan efek terlebih dahulu rights issue senilai rp197 3 triliun dan penerbitan obligasi rp104 4 triliun adapun dana yang dihimpun lewat pasar modal pada tahun lalu melonjak 206 persen dibandingkan 2020 simak video pilihan di bawah ini",

        '''
        # add newDict into data
        data.append(newDict)
    para_dict = {"title":"pre ipo data","paragraphs":data}
    data_dict = {"data":[para_dict]}
    final_data_for_model = {"root":data_dict}
    #print (final_data_for_model)
    
    #fileName = path.basename(csvFilePath).split(sep=".")[0] + "_data1.json"

    #B_DIR="squad"
    #newFileName = os.path.join(B_DIR,fileName)
    
    #fileName = path.basename(csvFilePath).split(sep=".")[0] + "_data1.json"\

    fileName = "squad/" + path.basename(csvFilePath).split(sep=".")[0] + "_data1.json"
    with open(fileName, "w") as td:
        json.dump(data_dict, td, indent=4, sort_keys=True)
        print("dump successful")


# fetch data from Multilex Table 
# put it to training.csv and test.csv ( 80:20 ratio) using train anf test model from sklearn in machine learning 
# convert the CSV to JSON - 
def create_datasets():
    df = fetch_data()
    train,test = train_test_split(df,test_size=0.2)
    train.to_csv("training.csv")
    test.to_csv("test.csv")
    #print(df.head())
    toJson("training.csv")
    toJson("test.csv")
    
    
#create_datasets()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement json (from versions: none)
ERROR: No matching distribution found for json
